In [ ]:
df_segments = pd.DataFrame({
            "SEGMENT": segments,
            column_name: [f"Decile {x}" for x in range(len(segments))]
        })

In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")


## Need a process for Looking at a .py File and then ensuring I have a reference usage here so I can remember.
## Need to also Classify the models based on usage and Dates so I can see changes.
## Need to determine a plan to migrate to a single usage.

In [ ]:
from data_d_strings import gpt_question
gpt_question('HuberRegressor')

'For the definition HuberRegressor can you please provide me a 4-5 sentences highlighting the definition, its origin, its importance and its application.'

In [ ]:
## FUNCTION TO STR
# Column Staistical Review Columns

In [20]:
from data_d_strings import template_doc_string_print

template_doc_string_print()



    Definition of Function

    Parameters:
        List of Parameters

    Returns:
        Object Type

    date_created:14-Jan-26
    date_last_modified: 14-Jan-26
    classification:TBD
    sub_classification:TBD
    usage:
        Example Function Call



In [ ]:
def PartitionDF(df, index_, func,include_all=True,*args,**kwargs):
    '''
    Function to automate the task of Applying a Specific Function on a dataframe which has been Segregatted based on 
    a combination of Columns as defined in the argument index_
    
    An example, where the function was originally designed: PartitionDF(temp_df1,['ENTITY','MEMBER_DURATION'],DFStatisticalReview) 
    
    Noting that to generate 32 distinct subsets on 50K Records and 124 Columns 6.25M records is 60 Seconds.
    
    Parameters:
        df (DataFrame): 
        index_ (list): List of Columns which will be included in Group By 
        func (Function): Function which will be applied to the function
        include_all (Bol): Option to include a Aggregated Calculation in Dataset, represented of calculation for All
        *args: Arguements for inclusion in Function
        **kwargs: Keyword Arguments for inclusion in Function (if required)
        
        
    Returns:
        Dictionary with Unique DataFrame and Indicataors for the record place and [FUNC] output.
        
    Date Created: August 20, 2025
    Date Last Modified: August 22, 2025. Added ValueError to Stop when DF missing Index Values, which could not be processed
    
    '''
    
    # Can not Partion Null Values. Dangerous to Assume. Return Error, force User to Solve Data Problems
    
    for record in index_:
        if len(df[df[record].isnull()])>0:
            raise ValueError(f'Null Value found in Index {record}, please review data. Fill or Remove Value')
    
    partitioned_list = []
    
    # Full dataset (All for all index columns)
    
    if include_all:
    
        temp_ = {value: 'All' for value in index_}
        temp_['DF'] = df
        temp_func = func(df).reset_index().rename(columns={'index': "COLUMN_NAME"})
        for column in index_:
            temp_func[column] = 'All'
        temp_['FUNC'] = temp_func
        partitioned_list.append(temp_)
    else:
        temp_ = dict()

    # Generate all partial combinations of index columns
    for i in range(1, len(index_)):
        for combo in itertools.combinations(index_, i):
            gb = df.groupby(list(combo))
            for selection in gb.groups.keys():
                temp_df = gb.get_group(selection).copy()
                if include_all:
                    temp_ = {col: 'All' for col in index_}
                if isinstance(selection, tuple):
                    for col, val in zip(combo, selection):
                        temp_[col] = val
                else:
                    temp_[combo[0]] = selection

                temp_['DF'] = temp_df
                temp_func = func(temp_df).reset_index().rename(columns={'index': "COLUMN_NAME"})
                for col in index_:
                    temp_func[col] = temp_[col]
                temp_['FUNC'] = temp_func
                partitioned_list.append(temp_)
    
    # Fully segmented combinations
    gb = df.groupby(index_)
    for selection in gb.groups.keys():
        temp_df = gb.get_group(selection).copy()
        temp_ = dict(zip(index_, selection))
        temp_['DF'] = temp_df
        temp_func = fun                  

def SummarizedDataSetforBITool(df, dimensions, metrics,apply_tranpose=False):
    """
    Builds a DataFrame with all combinations of ALL-level rollups 
    across the specified dimensions and metrics.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        dimensions (list of str): Dimension column names.
        metrics (list of str): Metric column names to aggregate.

    Returns:
        pd.DataFrame: Aggregated DataFrame with 'ALL' rollups.
        
        
    Date Created:
    Date Last Maintained: 26-Sep-25
    Updated Apply Transpose, to include Option to Transpose DF.
        
    """
    result_frames = []
    
    available_metrics = [x for x in metrics if x in df.columns]

    for r in range(len(dimensions) + 1):
        for dims in itertools.combinations(dimensions, r):
            group_cols = list(dims)
            
            # Aggregate metrics with or without groupby
            if group_cols:
                agg_df = df.groupby(group_cols, dropna=False)[available_metrics].sum().reset_index()
            else:
                # Grand total (ALL for all dims)
                sums = df[available_metrics].sum().to_frame().T
                agg_df = sums
                for col in dimensions:
                    agg_df[col] = 'ALL'

            # Fill missing dimension columns with 'ALL'
            for col in dimensions:
                if col not in group_cols:
                    agg_df[col] = 'ALL'

            # Ensure consistent column order
            agg_df = agg_df[dimensions + available_metrics]
            result_frames.append(agg_df)

    final_df = pd.concat(result_frames, ignore_index=True)
    
    if apply_tranpose:
        return TranposeDF(final_df,dimensions).rename(columns={'variable':'METRIC','value':"VALUE"})
    else:
        return final_df


def DFColumnManualSortOrder(df,column_name,column_order):
    
    '''
    
    
    '''
    
    df[column_name] = pd.Categorical(df[column_name], categories=column_order, ordered=True)

In [ ]:
# Removed Multithread Processing from Read Directory. Need to have displine and Principles related to 
# Creation of Functions, Create Individual functions which do components, opposed to long ones. 

In [13]:
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'
df = pd.read_csv(google_definition_csv)

In [14]:
df['Model'].value_counts()

Model
Linear                      4
Denisty Based Clustering    2
Graph-based clustering      2
Instance Based Learning     1
Tree                        1
Meta-Model                  1
Probabilistic clustering    1
Name: count, dtype: int64

In [ ]:
df['Algorithm Class'].value_counts()

In [ ]:
df['Learning Type'].value_counts()

In [ ]:
df['Categorization'].value_counts()

In [ ]:


def ReviewEntireDataframe(df,file_name=None):
    
    final_df = pd.DataFrame()
    
    for column in df.columns:
        start_time = timeit.default_timer()
        temp_df = ColumnStatisticalReview(df,column)
        print(f'Elapsed time to process {column}:{timeit.default_timer() - start_time:,.2f}')
        final_df = pd.concat([final_df,temp_df],axis=1)
    if file_name:
        final_df.to_csv(f"{file_name}.csv")
        
    return final_df


In [19]:
def CompareFunction(func1,func2,additional_records=20):
    
    '''
    Function which Compares 2 Functions and determines if they are different. Specifically, it can help to easily
    Manage Version control of Functions outside of a More robust environment such as GIT.
    

    '''
    
    list1 = FunctionToSTR(func1)
    list2 = FunctionToSTR(func2)
    
    length = max(len(list1),len(list2))
    
    for record in range(0,length):
        if list1[record]==list2[record]:
            if record == (length-1):
                print("All Records Reconcile")
            pass
        else:
            try:
                print(list1[record:record+additional_records])
                print(list2[record:record+additional_records])
            except:
                print(list1[record:record:])
                print(list2[record:record:])
            break


CompareFunction(test,TranposeNonTimeSeriesDF)

NameError: name 'FunctionToSTR' is not defined

In [17]:
def test(df, index, columns=None):
    '''
    Transposes a non-time-series DataFrame from wide to long format by melting specified columns.

    This is especially useful for flattening columns into a single column to support tools 
    like Power BI, where long format enables dynamic pivoting and aggregation.

    Parameters:
        df (DataFrame): The input pandas DataFrame.
        index (list): Columns to retain as identifiers (will remain unchanged).
        columns (list): Columns to unpivot into key-value pairs.

    Returns:
        DataFrame: A long-format DataFrame with 'variable' and 'value' columns.
    '''
    if not columns:
        columns = [col for col in final_df1.columns if (isinstance(col, pd.Timestamp))|(isinstance(col, datetime.datetime))]
    
    return df.melt(id_vars=index, value_vars=columns)


In [18]:
def TranposeNonTimeSeriesDF(df, index, columns=None):
    '''
    Transposes a non-time-series DataFrame from wide to long format by melting specified columns.

    This is especially useful for flattening columns into a single column to support tools 
    like Power BI, where long format enables dynamic pivoting and aggregation.

    Parameters:
        df (DataFrame): The input pandas DataFrame.
        index (list): Columns to retain as identifiers (will remain unchanged).
        columns (list): Columns to unpivot into key-value pairs.

    Returns:
        DataFrame: A long-format DataFrame with 'variable' and 'value' columns.
    '''
    if not columns:
        columns = [col for col in final_df1.columns if (isinstance(col, pd.Timestamp))|(isinstance(col, datetime.datetime))]
    
    return df.melt(id_vars=index, value_vars=columns)


## TIME SERIES

In [ ]:


import pandas as pd
import numpy as np

def TranposeNonTimeSeriesDF(df, index, columns=None):
    '''
    Transposes a non-time-series DataFrame from wide to long format by melting specified columns.

    This is especially useful for flattening columns into a single column to support tools 
    like Power BI, where long format enables dynamic pivoting and aggregation.

    Parameters:
        df (DataFrame): The input pandas DataFrame.
        index (list): Columns to retain as identifiers (will remain unchanged).
        columns (list): Columns to unpivot into key-value pairs.

    Returns:
        DataFrame: A long-format DataFrame with 'variable' and 'value' columns.
    '''
    if not columns:
        columns = [col for col in final_df1.columns if (isinstance(col, pd.Timestamp))|(isinstance(col, datetime.datetime))]
    
    return df.melt(id_vars=index, value_vars=columns)

def CreatePivotTableFromTimeSeries(df,
                                   index,
                                   columns,
                                   values,
                                   aggfunc='sum',
                                   skipna=True):
    
    '''
    Function to Summaryize a Time Series Dataframe into a Pivot. Creating a number of critical Metrics.
    
    
    
    '''
    
    # 1. Pivot
    if index==None:
        df1 = df.pivot_table(columns=columns,values=values,aggfunc=aggfunc)
    else:
        df1 = df.pivot_table(index=index, columns=columns, values=values, aggfunc=aggfunc)

    # 2. Capture original month columns IMMEDIATELY after pivot
    month_cols = df1.columns.tolist()
 
    # 3. Add rolling window stats
    if len(month_cols) >= 3:
        df1['AVG_3M'] = df1[month_cols[-3:]].mean(axis=1, skipna=skipna)
        df1['CHG_3M'] = df1[month_cols[-1]]-df1[month_cols[-3]]
        try:
            df1['PERC_CHG_3M'] = df1['CHG_3M']/df1[month_cols[-3]]
        except:
            df1['PERC_CHG_3M'] = 0
    
    if len(month_cols) >= 6:
        df1['AVG_6M'] = df1[month_cols[-6:]].mean(axis=1, skipna=skipna)
        df1['CHG_6M'] = df1[month_cols[-1]]-df1[month_cols[-6]]
        try:
            df1['PERC_CHG_6M'] = df1['CHG_6M']/df1[month_cols[-6]]
        except:
            df1['PERC_CHG_6M'] = 0
            
    if len(month_cols) >= 12:
        df1['AVG_12M'] = df1[month_cols[-12:]].mean(axis=1, skipna=skipna)
        df1['CHG_12M'] = df1[month_cols[-1]]-df1[month_cols[-12]]
        try:
            df1['PERC_CHG_12M'] = df1['CHG_12M']/df1[month_cols[-12]]
        except:
            df1['PERC_CHG_12M'] = 0

    df1['CHG_DF']  = df1[month_cols[-1]]-df1[month_cols[0]]
    df1['AVG_DF'] = df1[month_cols[-1:]].mean(axis=1, skipna=skipna)
    df1['PERC_CHG_DF'] = df1['AVG_DF']/df1[month_cols[-1]]

    
    # 4. Now calculate global stats **only using the original month columns**
    stats = pd.DataFrame({
        'MEAN': df1[month_cols].mean(axis=1, skipna=skipna),
        'STD': df1[month_cols].std(axis=1, skipna=skipna),
        'MAX': df1[month_cols].max(axis=1, skipna=skipna),
        'MIN': df1[month_cols].min(axis=1, skipna=skipna),
        'COUNT': df1[month_cols].count(axis=1)
    })

    # 5. Merge the stats
    df1 = pd.concat([df1, stats], axis=1)
    
    return df1.fillna(0)


def CreateMultiplePivotTableFromTimeSeries(df,
                                           index_list,
                                           metric_list,
                                           column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    

    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    
    for metric in metric_list:
        print(f'Attempting to Process:{metric}')
        try:
            all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
            cols = list(all_df.columns)
            all_df = all_df.reset_index(drop=True)
            all_df['METRIC'] = metric
            cols.insert(0,'METRIC')

            for key in index_list:
                cols.insert(0,key)
                all_df[key] = 'All'

            final_df = pd.concat([final_df,all_df[cols]])
            # Iterate through all Index Items Individually
            for key in index_list:
                temp = CreatePivotTableFromTimeSeries(df,
                                                      index=key,
                                                      values=metric,
                                                      columns=column).reset_index() 
                for missing in [x for x in index_list if x != key]:
                    temp[missing] = 'All'
                temp['METRIC'] = metric
                final_df = pd.concat([final_df,temp])

            # Add Value for Metric with Entire Index Combination
            temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        except:
            print(f'Could Not Process Metric:{metric}.')

    return final_df




def CreateMultiplePivotTableFromTimeSeries(df,index_list,metric_list,column):
    '''
    Function to utilize when Attempting to Create Multip[le Times Series. Specifically Multiple Metrics, and Multiple Index's
    



    '''
    
    final_df = pd.DataFrame()
    
    # Iterate through all Possible Metrics Selected.
    for metric in metric_list:
        all_df = CreatePivotTableFromTimeSeries(df=df,index=None,columns=column,values=metric,aggfunc='sum') 
        cols = list(all_df.columns)
        all_df = all_df.reset_index(drop=True)
        all_df['METRIC'] = metric
        cols.insert(0,'METRIC')

        for key in index:
            cols.insert(0,key)
            all_df[key] = 'All'

        final_df = pd.concat([final_df,all_df[cols]])

        # Iterate through all Index Items Individually
        for key in index_list:
            temp = CreatePivotTableFromTimeSeries(df,index=key,
                                                  values=metric,
                                                  columns=column).reset_index() 
            for missing in [x for x in index if x != key]:
                temp[missing] = 'All'
            temp['METRIC'] = metric
            final_df = pd.concat([final_df,temp])
        
        # Add Value for Metric with Entire Index Combination
        temp = CreatePivotTableFromTimeSeries(df,index=index_list,values=metric,columns=column).reset_index()
        temp['METRIC'] = metric
        final_df = pd.concat([final_df,temp])
    
    return final_df


def SummarizeTimeSeriesDf(df,
                          summary_cols,
                          primary_key_list):
    '''
    Function to Summarize a Time Series dataframe based on a finite number of identified Columns.
    
    Parameters
        df (Dataframe): TimeSeries in Nature
        summary_cols (List): List of Columns which are to be included in SUmmary
        primary_key_list (list): Primary Key of Dataframe
    
    Returns
        temp_df1: Raw Data of SUmmary Cols with a Count of Observations. If include Month Variable Easy to add to Pivot Table
        summary: Summary (Excluding Primary Key). including Total Observations, MEan, Max, Min.
    
    
    '''
    temp_df = df[summary_cols].copy()
    temp_df['COUNT'] = 1
    
    # Unique Occurances by Pivot Criteria. Important to Include Month
    temp_df1 = temp_df.groupby(summary_cols).sum().reset_index().rename(columns={'COUNT':'TOTAL_DAYS'})
    
    pivot_columns1 = [x for x in summary_cols if x not in primary_key_list]
    
    summary = temp_df1.groupby(pivot_columns1).agg(
        TOTAL=('TOTAL_DAYS', 'count'),
        AVG_DAYS_OPEN=('TOTAL_DAYS', 'mean'),
        MAX_OBS=('TOTAL_DAYS', 'min'),
        MIN_OBS=('TOTAL_DAYS', 'max')).reset_index()
    
    return temp_df1,summary


import pandas as pd
import numpy as np


## Recently Created Perhaps a Better Function. Review What I did.

In [ ]:
def single_column_compare(df,
                          column_name,
                          column_name1,
                          additional_filter=None,
                          bracketing=[-10000,-1000,-100,-1,0,1,100,1000,10000]):
    
    '''
    
    Function which takes a dataframe with 2 Columns which are identical and attempts to Compare.
    Function df_column_compare applies this function to an Entire Dataframe
    
    Parameters:
        column_name (str):
        column_name1 (str): For One Off Use, to compare different name
        additional_filter (str): Default parameter to distinguish combined dataframes, also used in MergeAndRenameColumnsDf
        bracketing(list): Value to create Distintion when Calculated Difference between columns is numeric.
    
    Returns:
        Dictionary of 3 Dataframes, Account, Summary and Group By.
    
    Values:
    
    
    Date Created: August 21, 2025
    Date Last Modified:
    


    '''
    if not column_name1:
        column_name1 = f"{column_name}{column_distinction}"    
    
    # Change Names of Individual Columns to Something Generic so datasets can be Concatenated.
    temp_df = df.rename(columns={column_name:'DF',column_name1:'DF1'}).copy()
    
    output_dict = {}
    
    temp_df['COLUMN_NAME'] = column_name
    
    BinaryComplexEquivlancey(temp_df,'DF','DF1','VALUES_EQUAL')
    
    temp_df['VALUES_NOT_EQUAL'] = np.where(temp_df['VALUES_EQUAL']==0,1,0)
    temp_df['NULL_RECORD_DF'] = np.where(temp_df['DF'].isnull(),1,0)
    temp_df['NULL_RECORD_DF1'] = np.where(temp_df['DF1'].isnull(),1,0)
    
    try:
        temp_df['DIFFERENCE'] = temp_df['DF'].fillna(0)-temp_df['DF1'].fillna(0)
    except:        
        temp_df['DIFFERENCE'] = 0
        
    try:
        BracketColumn(temp_df,'DIFFERENCE','DIFF_SEGMENT',bracketing)
    except:
        temp_df['DIFF_SEGMENT'] = 'Could Not Calculate'
    
    # Removed Column Partitioner as it wasn't being Used.
    
    temp_df1 = temp_df.copy()
    temp_df1['RECORD_COUNT']=1
    
    if additional_filter:
        output_dict['groupby_df'] = temp_df1[[additional_filter,'COLUMN_NAME','DF','DF1','RECORD_COUNT','VALUES_EQUAL','VALUES_NOT_EQUAL','NULL_RECORD_DF','NULL_RECORD_DF1']].groupby([additional_filter,'COLUMN_NAME','DF','DF1'],dropna=False).sum().sort_values('VALUES_EQUAL',ascending=False).head(20).reset_index()
        
    else:
        output_dict['groupby_df'] = temp_df1[['COLUMN_NAME','DF','DF1','RECORD_COUNT','VALUES_EQUAL','VALUES_NOT_EQUAL','NULL_RECORD_DF','NULL_RECORD_DF1']].groupby(['COLUMN_NAME','DF','DF1'],dropna=False).sum().sort_values('VALUES_EQUAL',ascending=False).head(20).reset_index()
    
    if additional_filter:
        summary_df = pd.DataFrame()
        for value in temp_df[additional_filter].unique():
            temp = temp_df[temp_df[additional_filter]==value]
            value_dict = {
                'Total Combined Records':len(temp),
                'Values Equal':temp['VALUES_EQUAL'].sum(),
                'Values Not Equal':len(temp[temp['VALUES_EQUAL']==0]),
                'Percent Values Equal': (temp['VALUES_EQUAL'].sum()/len(temp))*100,
                'Null Records DF':temp['NULL_RECORD_DF'].sum(),
                'Null Records DF1':temp['NULL_RECORD_DF1'].sum()}
            
            try:
                value_dict['Total Difference']=temp['DIFFERENCE'].sum()
            except:
                value_dict['Total Difference']=0
                
            sum_df = pd.DataFrame(value_dict.values(),index=value_dict.keys(),columns=[column_name]).T.reset_index().rename(columns={'index':"COLUMN_NAME"})
            sum_df[additional_filter] = value
            summary_df = pd.concat([summary_df,sum_df])
    else:
        value_dict = {
            'Total Combined Records':len(temp_df),
            'Values Equal':temp_df['VALUES_EQUAL'].sum(),
            'Values Not Equal':len(temp_df[temp_df['VALUES_EQUAL']==0]),
            'Percent Values Equal': (temp_df['VALUES_EQUAL'].sum()/len(temp_df))*100,
            'Null Records DF':temp['NULL_RECORD_DF'].sum(),
            'Null Records DF1':temp['NULL_RECORD_DF1'].sum()}
        
        try:
            value_dict['Total Difference']=temp['DIFFERENCE'].sum()
        except:
            value_dict['Total Difference']=0
            
        summary_df = pd.DataFrame(value_dict.values(),index=value_dict.keys(),columns=[column_name]).T.reset_index().rename(columns={'index':"COLUMN_NAME"})
        
    output_dict['summary_df'] = summary_df
    output_dict['account_df'] = temp_df
    
    return output_dict

def DfDqComparison(df,
                   primary_key_list,
                   additional_filter,
                   column_distinction='_',
                   bracketing=[-10000,-1000,-100,-1,0,1,100,1000,10000],
                   file_name=None):
    
    '''
    Function to Apply ColumnDQComparison against DataFrame.
    Assumes you start with a Dataframe with Multiple Columns Different only by Column Distinction.
    
    Parameters:
        df (DataFrame)
        primary_key_list (list): List of Primary Keys, which are REMOVE from comparison Loop.
        additional_filter (str): Filter Used to Create a distinct Dimension. Currently DOES NOT accept List
        column_distinction (str): String which is expected to compare Columns. Added as default with MergeIdenticalDF
        bracketing (list): Numbers which can be used to Calculate a Bracketed difference Column in COmparison
        file_name (str): If Included, it will generate Excel Copies (Excel Used as CSV had issues uploading to DF)
        
    Return:
        DataFrame of Groupby, Account and Summary calculations.
        
        Account: Listing of All Account Values, with Calculations
        Summary: A summary Calculation Speaking to Overall Comparison
        Groupby: List of Equivalent Values, to compare Material Record Change/Consistency
    
    Date Created: August 21, 2025
    Date Last Modified: 
    
    
    '''
    
    df = df.copy()
    
    # Only Need to Test Common Records Can do a Simple Dataframe Analysis on Non Common Records.
    
    #Iterate Through All Columns in Common to create Final Values.
    
    account_df = pd.DataFrame()
    groupby_df =  pd.DataFrame()
    summary_df = pd.DataFrame()

    for column_name in [x for x in df.columns if (x not in primary_key_list)&(x[-1]!=column_distinction)]:
        column_name1 = f"{column_name}{column_distinction}"
        try:
            temp_dict = IdenticalColumnDQValidation(df=df[['ACCTNBR',additional_filter,column_name,column_name1]],
                                                    column_name=column_name,
                                                    additional_filter=additional_filter,
                                                    bracketing=bracketing)
    
            account_df = pd.concat([account_df,temp_dict['account_df']])
            summary_df = pd.concat([summary_df,temp_dict['summary_df']])
            groupby_df = pd.concat([groupby_df,temp_dict['groupby_df']])
            
        except:
            print(f'Could Not Compute: {column_name}') 
            
    if file_name:
        account_df.to_csv(f"{file_name}_ACCOUNT.csv",index=False)
        summary_df.to_csv(f"{file_name}_SUMMARY.csv",index=False)
        groupby_df.to_csv(f"{file_name}_GROUPBY.csv",index=False)

    return account_df,summary_df,groupby_df


In [ ]:
def string_to_list(x):
    import ast
    if isinstance(x, str):
        return ast.literal_eval(x)
    return x

df['type'] = df['type'].apply(lambda x:string_to_list(x))
df['charged_moves'] = df['charged_moves'].apply(lambda x:string_to_list(x))
df['fast_moves'] = df['fast_moves'].apply(lambda x:string_to_list(x))

df.head(2)


def split_list_column_to_fixed_columns(
    df: pd.DataFrame,
    column: str,
    prefix: str = "Column"
) -> pd.DataFrame:
    """
    Split a DataFrame list column into a fixed number of positional columns.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame
    column : str
        Column containing list values
    max_columns : int
        Number of output columns to create
    prefix : str
        Prefix for column names (default 'Column')

    Returns
    -------
    pd.DataFrame
        DataFrame with fixed positional columns
    """

    max_columns = df['type'].apply(lambda x:len(x)).max()
    
    new_cols = [
        f"{prefix}{i+1}"
        for i in range(max_columns)
    ]

    expanded = pd.DataFrame(
        df[column].apply(
            lambda x: x[:max_columns] + [None] * (max_columns - len(x))
            if isinstance(x, list)
            else [None] * max_columns
        ).tolist(),
        columns=new_cols,
        index=df.index
    )

    return expanded



def expand_list_column_to_columns(
    df: pd.DataFrame,
    column: str,
    prefix: str = None,
    fill_value: int = 0
) -> pd.DataFrame:
    """
    Expand a DataFrame column containing lists into unique indicator columns.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame
    column : str
        Column containing list values
    prefix : str, optional
        Optional prefix for new columns
    fill_value : int
        Value for absence (default 0)

    Returns
    -------
    pd.DataFrame
        New DataFrame with one column per unique item
    """
    # Step 1: Explode into long format
    exploded = df[[column]].explode(column)

    # Step 2: Create indicator column
    exploded["_value"] = 1

    # Step 3: Pivot to wide format
    wide = (
        exploded
        .pivot_table(
            index=exploded.index,
            columns=column,
            values="_value",
            fill_value=fill_value
        )
    )

    # Optional prefix
    if prefix:
        wide = wide.add_prefix(f"{prefix}_")

    return wide.reset_index(drop=True)

def unique_items_from_list_column(df, column):
    """
    Extract unique items from a DataFrame column containing lists.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame
    column : str
        Column name containing list values

    Returns
    -------
    list
        Sorted list of unique items
    """
    return sorted({item for sublist in df[column] for item in sublist})